# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Wolf', 'Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

K. Kreckel  ->  K. Kreckel  |  ['K. Kreckel']
P. Garcia  ->  A. P. Garcia  |  ['P. Garcia']
J. Li  ->  J. Li  |  ['J. Li']
H.-W. Rix  ->  H.-W. Rix  |  ['H.-W. Rix']
J. Li  ->  J. Li  |  ['J. Li']
K. Kreckel  ->  K. Kreckel  |  ['K. Kreckel']
M. Benisty  ->  M. Benisty  |  ['M. Benisty']
Arxiv has 56 new papers today
          4 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/4 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2405.14943


extracting tarball to tmp_2405.14943...

 done.


K. Kreckel  ->  K. Kreckel  |  ['K. Kreckel']
J. Li  ->  J. Li  |  ['J. Li']
H.-W. Rix  ->  H.-W. Rix  |  ['H.-W. Rix']


list index out of range
Retrieving document from  https://arxiv.org/e-print/2405.14980


extracting tarball to tmp_2405.14980... done.
Retrieving document from  https://arxiv.org/e-print/2405.15102


extracting tarball to tmp_2405.15102...

 done.
Retrieving document from  https://arxiv.org/e-print/2405.15611
extracting tarball to tmp_2405.15611... done.


/opt/hostedtoolcache/Python/3.9.19/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/tmp/ipykernel_2257/1211882699.py:51: LatexWarning: 2405.15102 did not run properly
'PosixPath' object is not subscriptable
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2405.14943-b31b1b.svg)](https://arxiv.org/abs/2405.14943) | **SDSS-V Local Volume Mapper (LVM): A Glimpse into Orion**  |
|| <mark>K. Kreckel</mark>, et al. -- incl., <mark>P. Garcia</mark>, <mark>J. Li</mark>, <mark>H.-W. Rix</mark> |
|*Appeared on*| *2024-05-27*|
|*Comments*| *12 pages, 9 figures, submitted to A&A*|
|**Abstract**|            The Orion Molecular Cloud complex, one of the nearest (D~400 pc) and most extensively studied massive star-forming regions, is ideal for constraining the physics of stellar feedback, but its ~12 deg diameter on the sky requires a dedicated approach to mapping ionized gas structures within and around the nebula. The Sloan Digital Sky Survey (SDSS-V) Local Volume Mapper (LVM) is a new optical integral field unit (IFU) that will map the ionized gas within the Milky Way and Local Group galaxies, covering 4300 deg^2 of the sky with the new LVM Instrument (LVM-I). We showcase optical emission line maps from LVM covering 12 deg^2 inside of the Orion belt region, with 195,000 individual spectra combined to produce images at 0.07 pc (35.3 arcsec) resolution. This is the largest IFU map made (to date) of the Milky Way, and contains well-known nebulae (the Horsehead Nebula, Flame Nebula, IC 434, and IC 432), as well as ionized interfaces with the neighboring dense Orion B molecular cloud. We resolve the ionization structure of each nebula, and map the increase in both the [SII]/Ha and [NII]/Ha line ratios at the outskirts of nebulae and along the ionization front with Orion B. [OIII] line emission is only spatially resolved within the center of the Flame Nebula and IC 434, and our ~0.1 pc scale line ratio diagrams show how variations in these diagnostics are lost as we move from the resolved to the integrated view of each nebula. We detect ionized gas emission associated with the dusty bow wave driven ahead of the star sigma Orionis, where the stellar wind interacts with the ambient interstellar medium. The Horsehead Nebula is seen as a dark occlusion of the bright surrounding photo-disassociation region. This small glimpse into Orion only hints at the rich science that will be enabled by the LVM.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2405.14980-b31b1b.svg)](https://arxiv.org/abs/2405.14980) | **Active Galactic Nuclei and Host Galaxies in COSMOS-Web. II. First Look at the Kpc-scale Dual and Offset AGN Population**  |
|| <mark>J. Li</mark>, et al. |
|*Appeared on*| *2024-05-27*|
|*Comments*| *20 pages, 8 figures, submitted to ApJ*|
|**Abstract**|            Kpc-scale dual and offset Active Galactic Nuclei (AGNs) are signposts of accreting supermassive black holes (SMBHs) triggered during late-stage galaxy mergers, offering crucial insights into the coevolution of SMBHs and galaxies. However, robustly confirmed systems at high redshift (e.g., $z>1$) are scarce and biased towards the most luminous and unobscured systems. In this study, we systematically search for kpc-scale (projected separation $<15$ kpc) dual and offset AGNs around 571 moderate-luminosity, X-ray-selected AGNs including the obscured population, utilizing deep HST ACS/F814W and multiband JWST NIRCam imaging from the COSMOS-Web survey. We identify 59 dual and 30 offset AGN candidates in late stage major mergers based on spatially-resolved spectral energy distribution analyses. This translates to $\sim28$ and $\sim10$ bona-fide dual and offset AGNs using a probabilistic pair counting scheme to minimize chance superpositions. Notably, the fraction of dual and offset AGNs among moderate-luminosity ($L_{\rm bol}\sim10^{43}-10^{46}\ \rm erg\ s^{-1}$), obscured AGNs is nearly two orders of magnitude higher than that of the most luminous, unobscured quasar pairs. We find tentative evidence for an increasing pair fraction among AGNs with redshift (from a few percent at $z\sim 0.5$ to $\sim22.9_{-17.7}^{+27.5}\%$ at $z\sim4.5$) and a higher occurrence rate of dual over offset AGNs. There is no pileup of dual/offset AGNs below $\sim 2~{\rm kpc}$ separations. These results generally align with predictions from the ASTRID and Horizon-AGN cosmological simulations when matching sample selection criteria, implying a high probability of both BHs being active simultaneously in late-stage major mergers.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2405.15611-b31b1b.svg)](https://arxiv.org/abs/2405.15611) | **Planet-driven spirals in protoplanetary discs: limitations of the semi-analytical theory for observations**  |
|| D. Fasano, et al. -- incl., <mark>M. Benisty</mark> |
|*Appeared on*| *2024-05-27*|
|*Comments*| *10 pages, 7 figures. Submitted for publication in A&A March 22, 2024. After first iteration with the referee, resubmitted May 24, 2024*|
|**Abstract**|            Detecting protoplanets during their formation stage is an important but elusive goal of modern astronomy. Kinematic detections via the spiral wakes in the gaseous disc are a promising avenue to achieve this goal. We aim to test the applicability to observations in the low and intermediate planet mass regimes of a commonly used semi-analytical model for planet induced spiral waves. In contrast with previous works which proposed to use the semi-analytical model to interpret observations, in this study we analyse for the first time both the structure of the velocity and density perturbations. We run a set of FARGO3D hydrodynamic simulations and compare them with the output of the semi-analytic model in the code wakeflow, which is obtained by solving Burgers' equation using the simulations as an initial condition. We find that the velocity field derived from the analytic theory is discontinuous at the interface between the linear and nonlinear regions. After 0.2 r$_p$ from the planet, the behaviour of the velocity field closely follows that of the density perturbations. In the low mass limit, the analytical model is in qualitative agreement with the simulations, although it underestimates the azimuthal width and the amplitude of the perturbations, predicting a stronger decay but a slower azimuthal advance of the shock fronts. In the intermediate regime, the discrepancy increases, resulting in a different pitch angle between the spirals of the simulations and the analytic model. The implementation of a fitting procedure based on the minimisation of intensity residuals is bound to fail due to the deviation in pitch angle between the analytic model and the simulations. In order to apply this model to observations, it needs to be revisited accounting also for higher planet masses.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2405.15102-b31b1b.svg)](https://arxiv.org/abs/2405.15102) | **The Fraction of Dust Mass in the Form of PAHs on 10-50 pc Scales in Nearby Galaxies**  |
|| J. Sutter, et al. -- incl., <mark>K. Kreckel</mark> |
|*Appeared on*| *2024-05-27*|
|*Comments*| *Accepted at ApJ, 39 pages, 25 figures*|
|**Abstract**|            Polycyclic aromatic hydrocarbons (PAHs) are a ubiquitous component of the interstellar medium (ISM) in z~0 massive, star-forming galaxies and play key roles in ISM energy balance, chemistry, and shielding. Wide field of view, high resolution mid-infrared (MIR) images from JWST provides the ability to map the fraction of dust in the form of PAHs and the properties of these key dust grains at 10-50 pc resolution in galaxies outside the Local Group. We use MIR JWST photometric observations of a sample of 19 nearby galaxies from the "Physics at High Angular Resolution in Nearby GalaxieS" (PHANGS) survey to investigate the variations of the PAH fraction. By comparison to lower resolution far-IR mapping, we show that a combination of the MIRI filters (R$_{\rm{PAH}}$ = [F770W+F1130W]/F2100W) traces the fraction of dust by mass in the form of PAHs (i.e., the PAH fraction, or q$_{\rm{PAH}}$). Mapping R$_{\rm{PAH}}$ across the 19 PHANGS galaxies, we find that the PAH fraction steeply decreases in HII regions, revealing the destruction of these small grains in regions of ionized gas. Outside HII regions, we find R$_{\rm{PAH}}$ is constant across the PHANGS sample with an average value of 3.43$\pm$0.98, which, for an illuminating radiation field of intensity 2-5 times that of the radiation field in the solar neighborhood, corresponds to q$_{\rm{PAH}}$ values of 3-6%.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error 'PosixPath' object is not subscriptable</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2405.14943.md
    + _build/html/tmp_2405.14943/./f3.png
    + _build/html/tmp_2405.14943/./f2.png
    + _build/html/tmp_2405.14943/./f4.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\oiii}{[O \textsc{iii}]}$
$\newcommand{\nii}{[N \textsc{ii}]}$
$\newcommand{\sii}{[S \textsc{ii}]}$
$\newcommand{\oi}{[O \textsc{i}]}$
$\newcommand{\niion}{[N \textsc{i}]}$
$\newcommand{\hei}{[He \textsc{i}]}$
$\newcommand{\siii}{[S \textsc{iii}]}$
$\newcommand{\oii}{[O \textsc{ii}]}$
$\newcommand{\neii}{[Ne \textsc{ii}]}$
$\newcommand{\hii}{H \textsc{ii}}$
$\newcommand{\ha}{H\alpha}$
$\newcommand{\hb}{H\beta}$
$\newcommand{\cii}{[C \textsc{ii}]}$
$\newcommand{\revone}$
$\newcommand{\OSU}{\label{OSU} Department of Astronomy, The Ohio State University, 140 West 18th Avenue, Columbus, Ohio 43210, USA}$
$\newcommand{\Alberta}{\label{Alberta} Department of Physics, University of Alberta, Edmonton, AB T6G 2E1, Canada}$
$\newcommand{\ANU}{\label{ANU} Research School of Astronomy and Astrophysics, Australian National University, Canberra, ACT 2611, Australia}$
$\newcommand{\IPAC}{\label{IPAC} Caltech-IPAC, 1200 E. California Blvd. Pasadena, CA 91125, USA}$
$\newcommand{\Carnegie}{\label{Carnegie} Observatories of the Carnegie Institution for Science, 813 Santa Barbara Street, Pasadena, CA 91101, USA}$
$\newcommand{ÇAPP}{\label{CCAPP} Center for Cosmology and Astroparticle Physics, 191 West Woodruff Avenue, Columbus, OH 43210, USA}$
$\newcommand{\CfA}{\label{CfA}Harvard-Smithsonian Center for Astrophysics, 60 Garden Street, Cambridge, MA 02138, USA}$
$\newcommand{\CITEVA}{\label{CITEVA} Centro de Astronomía (CITEVA), Universidad de Antofagasta, Avenida Angamos 601, Antofagasta, Chile}$
$\newcommand{\CNRS}{\label{CNRS} CNRS, IRAP, 9 Av. du Colonel Roche, BP 44346, F-31028 Toulouse cedex 4, France}$
$\newcommand{\ESO}{\label{ESO} European Southern Observatory, Karl-Schwarzschild Stra{\ss}e 2, D-85748 Garching bei München, Germany}$
$\newcommand{\ESOChile}{\label{ESOChile} European Southern Observatory, Avenida Alonso de Cordoba 3107, Casilla 19, Santiago 19001, Chile}$
$\newcommand{\HD}{\label{HD} Astronomisches Rechen-Institut, Zentrum für Astronomie der Universität Heidelberg, Mönchhofstra\ss e 12-14, D-69120 Heidelberg, Germany}$
$\newcommand{\ICRAR}{\label{ICRAR} International Centre for Radio Astronomy Research, University of Western Australia, 35 Stirling Highway, Crawley, WA 6009, Australia}$
$\newcommand{\IRAM}{\label{IRAM} Institut de Radioastronomie Millimétrique (IRAM), 300 Rue de la Piscine, F-38406 Saint Martin d'Hères, France}$
$\newcommand{\ITA}{\label{ITA} Universität Heidelberg, Zentrum für Astronomie, Institut für Theoretische Astrophysik, Albert-Ueberle-Str 2, D-69120 Heidelberg, Germany}$
$\newcommand{\IWR}{\label{IWR} Universität Heidelberg, Interdisziplinäres Zentrum für Wissenschaftliches Rechnen, Im Neuenheimer Feld 205, D-69120 Heidelberg, Germany}$
$\newcommand{\JHU}{\label{JHU} Department of Physics and Astronomy, The Johns Hopkins University, Baltimore, MD 21218, USA}$
$\newcommand{\Leiden}{\label{Leiden} Leiden Observatory, Leiden University, P.O. Box 9513, 2300 RA Leiden, The Netherlands}$
$\newcommand{\Maryland}{\label{Maryland} Department of Astronomy, University of Maryland, College Park, MD 20742, USA}$
$\newcommand{\MPE}{\label{MPE} Max-Planck-Institut für extraterrestrische Physik, Giessenbachstra{\ss}e 1, D-85748 Garching, Germany}$
$\newcommand{\MPIA}{\label{MPIA} Max-Planck-Institut für Astronomie, Königstuhl 17, D-69117, Heidelberg, Germany}$
$\newcommand{\Nagoya}{\label{Nagoya} Department of Physics, Nagoya University, Furo-cho, Chikusa-ku, Nagoya, Aichi 464-8602, Japan}$
$\newcommand{\NRAO}{\label{NRAO} National Radio Astronomy Observatory, 520 Edgemont Road, Charlottesville, VA 22903-2475, USA}$
$\newcommand{\OAN}{\label{OAN} Observatorio Astronómico Nacional (IGN), C/Alfonso XII, 3, E-28014 Madrid, Spain}$
$\newcommand{\ObsParis}{\label{ObsParis} Sorbonne Université, Observatoire de Paris, Université PSL, CNRS, LERMA, F-75014, Paris, France}$
$\newcommand{\Princeton}{\label{Princeton} Department of Astrophysical Sciences, Princeton University, Princeton, NJ 08544 USA}$
$\newcommand{\UToledo}{\label{UToledo} University of Toledo, 2801 W. Bancroft St., Mail Stop 111, Toledo, OH, 43606}$
$\newcommand{\Toulouse}{\label{Toulouse} Université de Toulouse, UPS-OMP, IRAP, F-31028 Toulouse cedex 4, France}$
$\newcommand{\UBonn}{\label{UBonn} Argelander-Institut für Astronomie, Universität Bonn, Auf dem Hügel 71, 53121 Bonn, Germany}$
$\newcommand{\UChile}{\label{UChile} Departamento de Astronomía, Universidad de Chile, Camino del Observatorio 1515, Las Condes, Santiago, Chile}$
$\newcommand{\UConn}{\label{UConn} Department of Physics, University of Connecticut, Storrs, CT, 06269, USA}$
$\newcommand{\UCSD}{\label{UCSD}Center for Astrophysics and Space Sciences, Department of Physics,  University of California, San Diego, 9500 Gilman Drive, La Jolla, CA 92093, USA}$
$\newcommand{\UGent}{\label{UGent} Sterrenkundig Observatorium, Universiteit Gent, Krijgslaan 281 S9, B-9000 Gent, Belgium}$
$\newcommand{\ULyon}{\label{ULyon} Univ Lyon, Univ Lyon 1, ENS de Lyon, CNRS, Centre de Recherche Astrophysique de Lyon UMR5574,\ F-69230 Saint-Genis-Laval, France}$
$\newcommand{\UMass}{\label{UMass} University of Massachusetts—Amherst, 710 N. Pleasant Street, Amherst, MA 01003, USA}$
$\newcommand{\UWyoming}{\label{UWyoming} Department of Physics and Astronomy, University of Wyoming, Laramie, WY 82071, USA}$
$\newcommand{\LAM}{\label{LAM} Aix Marseille Univ, CNRS, CNES, LAM (Laboratoire d’Astrophysique de Marseille), Marseille, France}$
$\newcommand{\UHawaii}{\label{UHawaii} Institute for Astronomy, University of Hawaii, 2680 Woodlawn Drive, Honolulu, HI 96822, USA}$
$\newcommand{\UCM}{\label{UCM} Departamento de Física de la Tierra y Astrofísica, Universidad Complutense de Madrid, E-28040, Spain}$
$\newcommand{\IPARC}{\label{IPARC} Instituto de Física de Partículas y del Cosmos IPARCOS, Facultad de Ciencias Físicas, Universidad Complutense de Madrid, E-28040, Spain}$
$\newcommand{\STScI}{\label{STScI} Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\newcommand{\McMaster}{\label{McMaster} Department of Physics and Astronomy, McMaster University, 1280 Main Street West, Hamilton, ON L8S 4M1, Canada}$
$\newcommand{\INAF}{\label{INAF} INAF -- Osservatorio Astrofisico di Arcetri, Largo E. Fermi 5, I-50157, Firenze, Italy}$
$\newcommand{\Sydney}{\label{Sydney} Sydney Institute for Astronomy, School of Physics A28, The University of Sydney, NSW 2006, Australia}$
$\newcommand{\CITA}{\label{CITA} Canadian Institute for Theoretical Astrophysics (CITA), University of Toronto, 60 St George St, Toronto, ON M5S 3H8, Canada}$
$\newcommand{\ASIAA}{\label{ASIAA} Institute of Astronomy and Astrophysics, Academia Sinica, No. 1, Sec. 4, Roosevelt Road, Taipei 10617, Taiwan}$
$\newcommand{\TKU}{\label{TKU} Department of Physics, Tamkang University, No.151, Yingzhuan Rd., Tamsui Dist., New Taipei City 251301, Taiwan}$
$\newcommand{\PSMA}{\label{PSMA} Penn State Mont Alto, 1 Campus Drive, Mont Alto, PA  17237, USA}$
$\newcommand{\ILL}{\label{ILL} ILL}$
$\newcommand{\stromlo}{\label{stromlo} Research School of Astronomy and Astrophysics, Australian National University, Mt Stromlo Observatory, Weston Creek, ACT 2611, Australia}$
$\newcommand{\UCatolica}{\label{UCatolica} Instituto de Astronomía, Universidad Católica del Norte, Av. Angamos 0610, Antofagasta, Chile}$
$\newcommand{\UT}{\label{UT} McDonald Observatory, The University of Texas at Austin, 1 University Station, Austin, TX 78712-0259, USA}$
$\newcommand{\Vanderbilt}{\label{Vanderbilt} Department of Physics and Astronomy, Vanderbilt University, VU Station 1807, Nashville, TN 37235, USA}$
$\newcommand{\UNF}{\label{UNF} Department of Physics, University of North Florida, 1 UNF Dr. Jacksonville FL 32224}$
$\newcommand{\NAOC}{\label{NAOC} Chinese Academy of Sciences South America Center for Astronomy, National Astronomical Observatories, CAS, Beijing 100101, China}$
$\newcommand{\CASA}{\label{CASA} Center for Astrophysics and Space Astronomy, University of Colorado, 389 UCB, Boulder, CO 80309-0389, USA}$
$\newcommand{\UNAM}{\label{UNAM} Universidad Nacional Autónoma de México, Instituto de Astronomía, AP 106, Ensenada 22800, BC, México}$
$\newcommand{\UDP}{\label{UDP} Instituto de Estudios Astrofísicos, Facultad de Ingeniería y Ciencias, Universidad Diego Portales, Av. Ejército Libertador 441, Santiago, Chile}$
$\newcommand{\Steward}{\label{Steward} Steward Observatory, University of Arizona, 933 N. Cherry Ave., Tucson, AZ 85721-0065, USA}$
$\newcommand{\APO}{\label{APO} Apache Point Observatory and New Mexico State University, P.O. Box 59,$
$Sunspot, NM 88349-0059, USA}$
$\newcommand{\UNAMCU}{\label{UNAMCU} Universidad Nacional Autónoma de México, Instituto de Astronomía, AP 70-264, CDMX 04510, México}$
$\newcommand{\UWash}{\label{UWash}Department of Astronomy, University of Washington, Seattle, WA, 98195}$
$\newcommand{Ç}{\label{CC}Department of Physics, Colorado College, Colorado Springs, CO 80903}$
$\newcommand{\Utah}{\label{Utah}Department of Physics and Astronomy, University of Utah, 115 S. 1400 E., Salt Lake City, UT 84112, USA}$
$\newcommand{\UConcepcion}{\label{UConcepcion}Departamento de Astronomía, Universidad de Concepción, Casilla 160-C, Concepción, Chile}$
$\newcommand{\FCLA}{\label{FCLA}Franco-Chilean Laboratory for Astronomy, IRL 3386, CNRS and Universidad de Chile, Santiago, Chile}$
$\newcommand{\Oklahoma}{\label{Oklahoma}Homer L. Dodge Department of Physics and Astronomy, University of Oklahoma, Norman, OK 73019, USA}$
$\newcommand{\UIUC}{\label{UIUC}Department of Astronomy, University of Illinois, Urbana, IL 61801, USA}$
$\newcommand{\Harvard}{\label{Harvard}Harvard-Smithsonian Center for Astrophysics, Cambridge, MA 02138, USA}$
$\newcommand{\caltech}{\label{caltech}Department of Astronomy, California Institute of Technology, Pasadena, CA 91125, USA}$
$\newcommand\natexlab{#1}$</div>



<div id="title">

# SDSS-V Local Volume Mapper (LVM): A Glimpse into Orion

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2405.14943-b31b1b.svg)](https://arxiv.org/abs/2405.14943)<mark>Appeared on: 2024-05-27</mark> -  _12 pages, 9 figures, submitted to A&A_

</div>
<div id="authors">

<mark>K. Kreckel</mark>, et al. -- incl., <mark>J. Li</mark>, <mark>H.-W. Rix</mark>

</div>
<div id="abstract">

**Abstract:** The Orion Molecular Cloud complex, one of the nearest (D $\sim$ 400 pc) and most extensively studied massive star-forming regions, is ideal for constraining the physics of stellar feedback, but its $\sim$ 12 deg diameter on the sky requires a dedicated approach to mapping ionized gas structures within and around the nebula. The Sloan Digital Sky Survey (SDSS-V) Local Volume Mapper (LVM) is a new optical integral field unit (IFU) that will map the ionized gas within the Milky Way and Local Group galaxies, covering 4300 deg $^2$ of the sky with the new LVM Instrument. We showcase optical emission line maps from LVM covering 12 deg $^2$ inside of the Orion belt region, with 195,000 individual spectra combined to produce images at 0.07 pc (35.3 $\arcsec$ ) resolution. This is the largest IFU map made (to date) of the Milky Way, and contains well-known nebulae (the Horsehead Nebula, Flame Nebula, IC 434, and IC 432),  as well as ionized interfaces with the neighboring dense Orion B molecular cloud. We resolve the ionization structure of each nebula, and map the increase in both the $\sii$ / $\ha$ and $\nii$ / $\ha$ line ratios at the outskirts of nebulae and along the ionization front with Orion B. $\oiii$ line emission is only spatially resolved within the center of the Flame Nebula and IC 434, and our $\sim$ 0.1 pc scale line ratio diagrams show how variations in these diagnostics are lost as we move from the resolved to the integrated view of each nebula. We detect ionized gas emission associated with the dusty bow wave driven ahead of the star $\sigma$ Orionis, where the stellar wind interacts with the ambient interstellar medium. The Horsehead Nebula is seen as a dark occlusion of the bright surrounding photo-disassociation region. This small glimpse into Orion only hints at the rich science that will be enabled by the LVM.

</div>

<div id="div_fig1">

<img src="tmp_2405.14943/./f3.png" alt="Fig5" width="100%"/>

**Figure 5. -** Line emission and line ratios mapped across 20 pc$\times$20 pc of the Orion belt region. Top left: $\ha$ line emission is prominent within IC 434, the flame Nebula, and IC 432.
   Star symbols mark all individual ionizing O type (blue) and B type (cyan) stars, and dashed lines show the approximate boundaries of each nebula.  Missing individual pixels are masked fibers with bright stars where our simplistic stellar continuum fitting was insufficient.   Top center and right: $\sii$/$\ha$  and $\nii$/$\ha$ line ratio maps increase at the outskirts of IC 434 and the Flame nebulae. Bottom left: Dust extinction via the Balmer decrement increases along the extent of the neighboring Orion B molecular cloud, and peaks at the young embedded Flame Nebula. Bottom center: The increased dust concentration in the Flame Nebula is particularly pronounced in the $\sii$$\lambda$6717/$\sii$$\lambda$6731 line ratio, which traces changes in gas density.  Bottom right: $\oiii$/$\hb$ line ratios can only be mapped in the centers of IC 434 and the Flame Nebula. Higher values towards the center of IC 434 reflect the changing ionizing structure of the nebula, with more highly ionized oxygen found closer to the ionizing source.  (*fig:neb_conditions*)

</div>
<div id="div_fig2">

<img src="tmp_2405.14943/./f2.png" alt="Fig4" width="100%"/>

**Figure 4. -** A glimpse into Orion with LVM. 108 tiles cover 4 deg $\times$ 4 deg (30 pc $\times$ 30 pc), producing a total of almost 195,000 spectra. Colors show different emission lines ($\ha$: orange, $\sii$6717+6731: light blue, $\oiii$5007: magenta) trace the ionization structure of the nebula, and carve a bubble into surrounding dense gas (WISE 12$\mu$m: yellow). The ionized gas emission traces wispy, filamentary structures and dusty eroding clouds and clumps.  The iconic Horsehead Nebula sits at the bright filamentary ionization front and photo-disassociation region between the central ionized nebula IC 434 and the dark neighboring molecular cloud Orion B to the east, while  channels for escaping radiation are apparent in the north.    (*fig:orion_3color*)

</div>
<div id="div_fig3">

<img src="tmp_2405.14943/./f4.png" alt="Fig6" width="100%"/>

**Figure 6. -** Diagnostic line ratios (left: $\oiii$/$\hb$ vs $\nii$/$\ha$; right: $\oiii$/$\hb$ vs $\sii$/$\ha$) for all pixels with significant detections in all lines. The curves shown from \cite{Kewley2001}(solid) and \cite{Kauffmann2003}(dashed) are commonly used (in integrated and kpc regions) to distinguish photoionized gas from gas ionized by harder ionizing sources. Requiring the detection of $\oiii$  this diagnostic is limited to the areas of the Flame Nebula and the center of NGC 434 (see Figure \ref{fig:neb_conditions}), as well as to regions consistent with photoionization. Integrated results for IC 434 (triangle), IC 432 (square) and the Flame Nebula (circle) are overplotted, as well as integrated measurements from WHAM for single O-star powered Milky Way $\hii$ regions (orange) and extragalactic PHANGS $\hii$ regions with matched $\ha$ luminosity (purple). (*fig:bpt*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2405.14943"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

78  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

6  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

1  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
